In [2]:
import pandas as pd
import xarray as xr
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, Polygon, mapping
import os

In [3]:
base_path = 'C:\\Users\\Nayeli\\Documents\\Thesis'

In [4]:
oldest = pd.read_csv(base_path + '\\Response Functions\\No Adaptation\\oldest.csv')

### POP data

In [6]:
ssp = xr.open_dataset(f'D:\\data\\2_projection\\2_econ_vars\\SSP3.nc4')
ssp

<xarray.Dataset>
Dimensions:    (model: 2, region: 24378, year: 91)
Coordinates:
    ssp        object ...
  * model      (model) object 'high' 'low'
  * region     (region) object 'ABW' 'AFG.1.12' ... 'ZWE.9.R19a261b63a503c9c'
  * year       (year) int64 2010 2011 2012 2013 2014 ... 2097 2098 2099 2100
Data variables:
    pop        (model, region, year) float64 ...
    pop0to4    (model, region, year) float64 ...
    pop5to64   (model, region, year) float64 ...
    pop65plus  (model, region, year) float64 ...
    gdppc      (model, region, year) float64 ...
    gdp        (model, region, year) float64 ...

In [4]:
''' Generate files per scenario and age group for all impact regions '''

scenarios = ['SSP1', 'SSP2', 'SSP3', 'SSP4', 'SSP5']
groups = ['pop', 'pop0to4', 'pop5to64', 'pop65plus']
age_groups = ['total', 'young', 'older', 'oldest']

for scenario in scenarios:
    ssp = xr.open_dataset(f'D:\\data\\2_projection\\2_econ_vars\\{scenario}.nc4')
    for group, age_group in zip(groups, age_groups):
        ssp_pop = ssp[group].sel(model='low')
        df = ssp_pop.to_dataframe()
        df = df.drop(['ssp', 'model'], axis=1).unstack('year')
        df.columns = df.columns.get_level_values(1)
        df = df.reset_index()
        df['hierid'] = df['region']
        df = df.merge(oldest['region'], on='region', how='right')
        df.to_csv(os.path.join(base_path, f'GPD-POP\\POP files\\POP_{scenario}_{age_group}.csv'))
        print(f'{scenario} - {group}')

SSP1 - pop
SSP1 - pop0to4
SSP1 - pop5to64
SSP1 - pop65plus
SSP2 - pop
SSP2 - pop0to4
SSP2 - pop5to64
SSP2 - pop65plus
SSP3 - pop
SSP3 - pop0to4
SSP3 - pop5to64
SSP3 - pop65plus
SSP4 - pop
SSP4 - pop0to4
SSP4 - pop5to64
SSP4 - pop65plus
SSP5 - pop
SSP5 - pop0to4
SSP5 - pop5to64
SSP5 - pop65plus


In [18]:
'''Generate files of global young, older, oldest population per year'''

scenarios = ['SSP1', 'SSP2', 'SSP3', 'SSP5']
age_groups = ['young', 'older', 'oldest']
groups = ['pop0to4', 'pop5to64', 'pop65plus']
years = np.arange(2015,2105,5)

index = pd.MultiIndex.from_product([age_groups, scenarios], names=['Age group', 'Scenario'])
df = pd.DataFrame(index=index, columns=years)

for scenario in scenarios:
    ssp_pop = xr.open_dataset(f'D:\\data\\2_projection\\2_econ_vars\\{scenario}.nc4')
    for age_group, group in zip(age_groups, groups): 
        pop_vals = ssp_pop[group].sel(model='high').sum(dim='region')[::5].values
        df.loc[(age_group, scenario),:] = pop_vals[1:]
        
df = df.round(1)

df.to_csv('POP_global.csv')

### GDPpc data

In [5]:
### Generate High and Low GDP projections ###

scenarios = ['SSP1', 'SSP2', 'SSP3', 'SSP4', 'SSP5']

for scenario in scenarios:
    for model in ['low', 'high']:
        # Open xarray from Carleton data
        ssp = xr.open_dataset(f'D:\\data\\2_projection\\2_econ_vars\\{scenario}.nc4')
        # Select high or low model
        ssp = ssp.gdppc.sel(model=f'{model}')
        # Convert to dataframe
        df = ssp.to_dataframe()
        # Unstack
        df = df.drop(['ssp', 'model'], axis=1).unstack('year')
        df.columns = df.columns.get_level_values(1)
        # Reset index
        df = df.reset_index()
        df = df.merge(oldest['region'], on='region', how='right')
        df.to_csv(os.path.join(base_path, f'GPD-POP\\GDPpc files\\GDPpc_{scenario}_{model}.csv'))
        print(f'{scenario} - {model}')

SSP1 - low
SSP1 - high
SSP2 - low
SSP2 - high
SSP3 - low
SSP3 - high
SSP4 - low
SSP4 - high
SSP5 - low
SSP5 - high


In [7]:
for SSP in scenarios:
    SSP_GDP_high = pd.read_csv(os.path.join(base_path, f'GPD-POP\\GDPpc files\\GDPpc_{scenario}_high.csv'))
    SSP_GDP_low = pd.read_csv(os.path.join(base_path, f'GPD-POP\\GDPpc files\\GDPpc_{scenario}_low.csv'))
    SSP_GDP_mean = SSP_GDP_high.copy()
    for year in range(2010,2101):
        SSP_GDP_mean[f'{year}'] = (SSP_GDP_high[f'{year}'] + SSP_GDP_low[f'{year}']) / 2
    #SSP_GDP_mean.rename(columns={'region': 'hierid'}, inplace=True)
    SSP_GDP_mean.to_csv((os.path.join(base_path, f'GPD-POP\\GDPpc files\\GDPpc_{SSP}_mean.csv')))

### NSAT data 1 climate model

In [16]:
scenarios = ['SSP126', 'SSP245', 'SSP370', 'SSP585']
for scenario in scenarios:
    # Open one file of NSAT 
    ssp = pd.read_csv('C:\\Users\\Nayeli\\Documents\\Thesis\\CMIP\\NSAT SSP\\SSP126\\SSP126_2015.csv')
    # Create empty dataframe for yearly mean nsat
    mean_df = pd.DataFrame()
    # Apend once the hierid (region id)
    mean_df['region'] = ssp['hierid']
    # For for all years
    for year in range(2015, 2101):
        # Open yearly files produced for NSAT
        ssp_year = pd.read_csv(f'C:\\Users\\Nayeli\\Documents\\Thesis\\CMIP\\NSAT SSP\\{scenario}\\{scenario}_{year}.csv')
        # Keep only columns with temperature data
        columns_year = [col for col in ssp_year.columns if col.startswith(str(year))]
        # Take mean
        mean_df[f'{year}'] = ssp_year[columns_year].mean(axis=1)
    mean_df.to_csv(f'NSAT_{scenario}.csv')